# Processing - merging

In [ ]:
# packages
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy


curfolder = os.getcwd()
print(curfolder)

# files to work with
MTfolder = 'C:\\Users\\kadava\\Documents\\Github\\FLESH_3Dtracking_new\\projectdata\\' ## FLAGGED CHANGE
ACfolder = 'E:\\charade_experiment_WORKSPACE\\xdf_procedure\\data\\Data_processed\\Data_trials\\Audio_48\\' ## FLAGGED CHANGE
BBfolder = 'E:\\charade_experiment_WORKSPACE\\xdf_procedure\\data\\Data_processed\\Data_trials\\'

# folders to save the processed data
MTfolder_processed = curfolder + '\\TS_motiontracking\\'
ACfolder_processed = curfolder + '\\TS_acoustics\\'
TSmerged = curfolder + '\\TS_merged\\'

In [ ]:
import glob
import scipy
import pandas as pd

# prepare all files
bbfiles = glob.glob(MTfolder_processed + '/bb*.csv')
idfiles = glob.glob(MTfolder_processed + '/id*.csv')
ikfiles = glob.glob(MTfolder_processed + '/ik*.csv')
mtfiles = glob.glob(MTfolder_processed + '/mt*.csv')
mtfiles = [x for x in mtfiles if 'centered' not in x]
envfiles = glob.glob(ACfolder_processed + '/env*.csv')
f0files = glob.glob(ACfolder_processed + '/f0*.csv')
soundgen = glob.glob(ACfolder_processed + '/soundgen*.csv')
formants = glob.glob(ACfolder_processed + '/praat_formants*.csv')

desired_sr = 0.5    # this is the sr we are going to merge on (in Hz/sec)


for file in bbfiles:
    print('working on ' + file)
    bb_df = pd.read_csv(file)
    # get trial id
    trialid = bb_df['TrialID'][0]
    # find the same trialid in idfiles
    id_file = [x for x in idfiles if trialid in x]
    try:
        id_df = pd.read_csv(id_file[0])
    except IndexError:
        print('IndexError: ' + trialid + 'not found')
        continue
    
    # find the same trialid in mtfiles
    mt_file = [x for x in mtfiles if trialid in x]
    try:
        mt_df = pd.read_csv(mt_file[0])
    except IndexError:
        print('IndexError: ' + trialid + 'not found')
        continue
    # rename Time to time
    mt_df.rename(columns={'Time': 'time'}, inplace=True)

    # find the same trialid in envfiles
    env_file = [x for x in envfiles if trialid in x]
    try:
        env_df = pd.read_csv(env_file[0])
    except IndexError:
        print('IndexError: ' + trialid + 'not found')
        continue
    # rename trialID to TrialID
    env_df.rename(columns={'trialID': 'TrialID'}, inplace=True)

    # find the same trialid in f0files
    f0_file = [x for x in f0files if trialid in x]
    try:
        f0_df = pd.read_csv(f0_file[0])
    except IndexError:
        print('IndexError: ' + trialid + 'not found')
        continue

    # rename time_ms to time
    f0_df.rename(columns={'time_ms': 'time'}, inplace=True)
    # rename ID to TrialID
    f0_df.rename(columns={'ID': 'TrialID'}, inplace=True)

    # find the same trialid in ikfiles
    ik_file = [x for x in ikfiles if trialid in x]
    try:
        ik_df = pd.read_csv(ik_file[0])
    except IndexError:
        print('IndexError: ' + trialid + 'not found')
        continue

    # find the same trialid in soundgen
    soundgen_file = [x for x in soundgen if trialid in x]
    try:
        soundgen_df = pd.read_csv(soundgen_file[0])
    except IndexError:
        print('IndexError: ' + trialid + 'not found')
        continue
    
    # adapt the File column such that it has only last element separated by /
    soundgen_df['File'] = soundgen_df['File'].apply(lambda x: x.split('/')[-1])
    # get rid of .wav ending
    soundgen_df['File'] = soundgen_df['File'].apply(lambda x: x.split('.')[0])
    # replace Mic_nominal_srate48000_ by ''
    soundgen_df['File'] = soundgen_df['File'].apply(lambda x: x.replace('Mic_nominal_srate48000_', ''))
    # replace pr or trial by ''
    soundgen_df['File'] = soundgen_df['File'].apply(lambda x: x.replace('pr_', ''))
    soundgen_df['File'] = soundgen_df['File'].apply(lambda x: x.replace('trial_', ''))
    # if there is corrected, replace by ''
    soundgen_df['File'] = soundgen_df['File'].apply(lambda x: x.replace('_corrected', ''))
    # leave the first 4 elements
    soundgen_df['File'] = soundgen_df['File'].apply(lambda x: '_'.join(x.split('_')[:4]))

    # rename the col to TrialID
    soundgen_df.rename(columns={'File': 'TrialID'}, inplace=True)

    # leave only columns harmEnergy, f1_freq, f2_freq, loudness, novelty, roughness
    soundgen_df = soundgen_df[['time', 'TrialID', 'harmEnergy', 'loudness', 'novelty', 'roughness', 'flux']]

    # find the same trialid in formants
    formants_file = [x for x in formants if trialid in x]
    try:
        formants_df = pd.read_csv(formants_file[0])
    except IndexError:
        print('IndexError: ' + trialid + 'not found')
        continue

    # rename triald to TrialID
    formants_df.rename(columns={'trialid': 'TrialID'}, inplace=True)

    # keep only time, f1, f2, TrialID
    formants_df = formants_df[['time', 'f1', 'f2', 'f3', 'TrialID']]

    # convert time to ms
    formants_df['time'] = formants_df['time'] * 1000

    
    ######################################

    #### regularize sr in bb
    time_new = np.arange(0, max(bb_df['time']), 1/desired_sr)
    bb_interp = pd.DataFrame({'time': time_new})
    
    # interpolate all columns in samplebb except of the first and last
    colstoint = bb_df.columns
    # get rid of time and trialid
    colstoint = [x for x in colstoint if 'time' not in x]
    colstoint = [x for x in colstoint if 'TrialID' not in x]
    colstoint = [x for x in colstoint if 'FileInfo' not in x]

    for col in colstoint:
        bb_interp[col] = bb_df[col].interpolate(method='linear', x = bb_interp['time'])

    # add trialid and time
    bb_interp['TrialID'] = trialid
    bb_interp['FileInfo'] = bb_df['FileInfo'][0]
    
    ##### merge the bb_interp with env
    # merge the two dataframes
    merge1 = pd.merge(bb_interp, env_df, on=['time', 'TrialID'], how='outer')

    # interpolate missing values of envelope and audio
    colstoint = merge1.columns
    colstoint = [x for x in colstoint if 'audio' in x or 'envelope' in x]

    for col in colstoint: 
        merge1[col] = merge1[col].interpolate(method='linear', x = merge1['time'])

    # now we can kick out all vlaues where COPc is NaN
    merge1 = merge1[~np.isnan(merge1['COPc'])]

    #### merge with ID
    # merge the two dataframes
    merge2 = pd.merge(merge1, id_df, on=['time', 'TrialID'], how='outer')

    # get cols of sampleid
    colstoint = id_df.columns
    colstoint = [x for x in colstoint if 'time' not in x]
    colstoint = [x for x in colstoint if 'TrialID' not in x]

    # interpolate 
    for col in colstoint:
        merge2[col] = merge2[col].interpolate(method='linear', x = merge2['time'])

    # now we can kick out all values where COPc is NaN to get sampling rate back to 500	
    merge2 = merge2[~np.isnan(merge2['COPc'])]

    #### merge with MT
    # merge the two dataframes
    merge3 = pd.merge(merge2, mt_df, on=['time', 'TrialID'], how='outer')

    # get cols of samplemt
    colstoint = mt_df.columns
    colstoint = [x for x in colstoint if 'time' not in x]
    colstoint = [x for x in colstoint if 'TrialID' not in x]

    # interpolate missing values of from mt
    for col in colstoint:
        # interpolate
        merge3[col] = merge3[col].interpolate(method='linear', x = merge3['time'])

    # now we can kick out all values where COPc is NaN
    merge3 = merge3[~np.isnan(merge3['COPc'])]

    #### merge with F0
    # for interpolation, we need to again parse f0 into chunks of non-NaN values
    f0_df['chunk'] = None

    # annotate chunks of non-NaN values
    chunk = 0
    for index, row in f0_df.iterrows():
        if np.isnan(row['f0']):
            continue
        else:
            f0_df.loc[index, 'chunk'] = chunk
            # if the next value is NaN or this is the last row, increase the chunk
            if index == len(f0_df)-1:
                continue
            elif np.isnan(f0_df.loc[index+1, 'f0']):
                chunk += 1

    chunks = f0_df['chunk'].unique()

    # skip if chunks are empty (that means that there is no f0 trace)
    if len(chunks) > 1:
        # ignore the first chunk (None)
        chunks = chunks[1:]

    #now we can merge
    merge4 = pd.merge(merge3, f0_df, on=['time', 'TrialID'], how='outer')

    # we ignore the None chunk above, so if there is some trace, None should not be within chunks
    if None not in chunks:
        for chunk in chunks:
            # get the first and last row of the chunk
            firstrow = merge4[merge4['chunk'] == chunk].index[0]
            lastrow = merge4[merge4['chunk'] == chunk].index[-1]
            # fill all inbetween with the chunk number
            merge4.loc[firstrow:lastrow, 'chunk'] = chunk
            # get the rows of the chunk
            chunkrows = merge4[merge4['chunk'] == chunk]
            # interpolate
            chunkrows['f0'] = chunkrows['f0'].interpolate(method='linear', x = chunkrows['time'])
            # put the interpolated chunk back to the merge4
            merge4.loc[merge4['chunk'] == chunk, 'f0'] = chunkrows['f0']

    # get rid of the chunk column
    merge4.drop('chunk', axis=1, inplace=True)

    # now we can drop all rows where COPc is NaN
    merge4 = merge4[~np.isnan(merge4['COPc'])]

    ### merge with IK
    merge5 = pd.merge(merge4, ik_df, on=['time', 'TrialID'], how='outer')

    # get cols of sampleik
    colstoint = ik_df.columns
    colstoint = [x for x in colstoint if 'time' not in x]
    colstoint = [x for x in colstoint if 'TrialID' not in x]

    # interpolate missing values of from ik
    for col in colstoint:
        # interpolate
        merge5[col] = merge5[col].interpolate(method='linear', x = merge5['time'])

    # now we can kick out all values where COPc is NaN
    merge5 = merge5[~np.isnan(merge5['COPc'])]

    ### merge with soundgen
    # first we need to parse all cols on chunks

    # cols to chunk
    colstochunk = soundgen_df.columns
    colstochunk = [x for x in colstochunk if 'TrialID' not in x]
    colstochunk = [x for x in colstochunk if 'time' not in x]

    for col in colstochunk:
        soundgen_df[col + '_chunk'] = None

        chunk = 0
        for index, row in soundgen_df.iterrows():
            if np.isnan(row[col]):
                continue
            else:
                soundgen_df.loc[index, col + '_chunk'] = chunk
                # if the next value is NaN or this is the last row, increase the chunk
                if index == len(soundgen_df)-1:
                    continue
                elif np.isnan(soundgen_df.loc[index+1, col]):
                    chunk += 1

    # merge
    merge6 = pd.merge(merge5, soundgen_df, on=['time', 'TrialID'], how='outer')

    for col in colstochunk:
        # get the chunks
        chunks = merge6[col + '_chunk'].unique()
        # ignore chunk nan and None
        chunks = [x for x in chunks if x is not None]
        chunks = [x for x in chunks if x is not np.nan]
 
        # interpolate
        for chunk in chunks:
            # get the first and last row of the chunk
            firstrow = merge6[merge6[col + '_chunk'] == chunk].index[0]
            lastrow = merge6[merge6[col + '_chunk'] == chunk].index[-1]
            # fill all inbetween with the chunk number
            merge6.loc[firstrow:lastrow, col + '_chunk'] = chunk
            # get the rows of the chunk
            chunkrows = merge6[merge6[col + '_chunk'] == chunk]
            # interpolate , not putting any limits
            chunkrows[col] = chunkrows[col].interpolate(method='linear', x = chunkrows['time'])
            # put the interpolated chunk back to the merge6
            merge6.loc[merge6[col + '_chunk'] == chunk, col] = chunkrows[col]


    # get rid of the chunk columns
    merge6 = merge6.drop([x for x in merge6.columns if 'chunk' in x], axis=1)

    # now we can kick out all values where COPc is NaN
    merge6 = merge6[~np.isnan(merge6['COPc'])]

    ### merge with formants
    merge7 = pd.merge(merge6, formants_df, on=['time', 'TrialID'], how='outer')

    # get cols of sampleformants
    colstoint = formants_df.columns
    colstoint = [x for x in colstoint if 'time' not in x]
    colstoint = [x for x in colstoint if 'TrialID' not in x]

    # interpolate missing values of from formants
    for col in colstoint:
        # interpolate
        merge7[col] = merge7[col].interpolate(method='linear', x = merge7['time'])

    # now we can kick out all values where COPc is NaN
    merge7 = merge7[~np.isnan(merge7['COPc'])]

    # this is final df
    merge_final = merge7    

    ##### now we need to keep keep formants only in some intervals
    ### for that we first need to find out, where are peak widths for envelope peaks
    # find peaks in envelope, with min=mean
    peaks, _ = scipy.signal.find_peaks(merge_final['envelope'], height=np.mean(merge_final['envelope']))
    # get widths of the peaks
    widths = scipy.signal.peak_widths(merge_final['envelope'], peaks, rel_height=0.95)
    # peak width df with starts and ends
    peak_widths = pd.DataFrame({'start': widths[2], 'end': widths[3]})

    # now create a new column env_weak_width, and put 0s everywhere, and 1s in the intervals of the width
    merge_final['env_peak_width'] = 0
    for index, row in peak_widths.iterrows():
        merge_final.loc[int(row['start']):int(row['end']), 'env_peak_width'] = 1

    #### now we will create f1_clean and f2_clean, where we will keep only formants in the intervals of env_pak_width OR where f0 is not NaN
    merge_final['f1_clean_f0'] = merge_final['f1']
    merge_final['f2_clean_f0'] = merge_final['f2']
    merge_final['f3_clean_f0'] = merge_final['f3']

    # where f0 is NaN, we will put NaN
    merge_final.loc[np.isnan(merge_final['f0']), 'f1_clean_f0'] = np.nan
    merge_final.loc[np.isnan(merge_final['f0']), 'f2_clean_f0'] = np.nan
    merge_final.loc[np.isnan(merge_final['f0']), 'f3_clean_f0'] = np.nan

    #### we will also create f1_clean_env and f2_clean_env, where we will keep only formants in the intervals of env_pak_width
    merge_final['f1_clean_env'] = merge_final['f1']
    merge_final['f2_clean_env'] = merge_final['f2']
    merge_final['f3_clean_env'] = merge_final['f3']

    # where env_peak_width is 0, we will put NaN
    merge_final.loc[merge_final['env_peak_width'] == 0, 'f1_clean_env'] = np.nan
    merge_final.loc[merge_final['env_peak_width'] == 0, 'f2_clean_env'] = np.nan
    merge_final.loc[merge_final['env_peak_width'] == 0, 'f3_clean_env'] = np.nan

    ## now create f1_clean and f2_clean, where we copy values from clean_env and clean_f0
    merge_final['f1_clean'] = merge_final['f1_clean_env']
    merge_final['f2_clean'] = merge_final['f2_clean_env']
    merge_final['f3_clean'] = merge_final['f3_clean_env']

    # copy also f1_clean_f0 and f2_clean_f0
    merge_final.loc[np.isnan(merge_final['f1_clean']), 'f1_clean'] = merge_final['f1_clean_f0']
    merge_final.loc[np.isnan(merge_final['f2_clean']), 'f2_clean'] = merge_final['f2_clean_f0']
    merge_final.loc[np.isnan(merge_final['f3_clean']), 'f3_clean'] = merge_final['f3_clean_f0']

    # now for f1_clean and f2_clean, calculate velocity (and smooth
    merge_final['f1_clean_vel'] = np.insert(np.diff(merge_final['f1_clean']), 0, 0)
    merge_final['f2_clean_vel'] = np.insert(np.diff(merge_final['f2_clean']), 0, 0)
    merge_final['f3_clean_vel'] = np.insert(np.diff(merge_final['f3_clean']), 0, 0)

    # smooth
    merge_final['f1_clean_vel'] = scipy.signal.savgol_filter(merge_final['f1_clean_vel'], 5, 3)
    merge_final['f2_clean_vel'] = scipy.signal.savgol_filter(merge_final['f2_clean_vel'], 5, 3)
    merge_final['f3_clean_vel'] = scipy.signal.savgol_filter(merge_final['f3_clean_vel'], 5, 3)

    ####### calculate power
    # get all columns that contain 'torque_sum'
    torquestosum = [x for x in merge_final.columns if 'torque_sum' in x]
    # but not change
    torquestosum = [x for x in torquestosum if 'change' not in x]

    # get all columns that contain 'angSpeed_sum'
    angSpeedtosum = [x for x in merge_final.columns if 'angSpeed_sum' in x]

    groups = ['lowerbody', 'leg', 'head', 'arm']

    for group in groups:
        # get all columns that contain group
        cols = [x for x in merge_final.columns if group in x]

        # get all columns that contain 'torque_sum'
        torque = [x for x in cols if 'torque_sum' in x]
        # but not change
        torque = [x for x in torquestosum if 'change' not in x][0]

        # get all columns that contain 'angSpeed_sum'
        angSpeed = [x for x in cols if 'angSpeed_sum' in x][0]

        # get power which is torque * angSpeed
        merge_final[group + '_power'] = merge_final[torque] * merge_final[angSpeed]
        # smooth
        merge_final[group + '_power'] = scipy.signal.savgol_filter(merge_final[group + '_power'], 15,3)
    
    #print(merge_final)
    #break
    #write to csv
    merge_final.to_csv(TSmerged + '/merged_' + trialid + '.csv', index=False)    

In [ ]:
# mergedfiles
mergedfiles = glob.glob(TSmerged + '/merged*.csv')

# load random file
samplemerged = pd.read_csv(mergedfiles[20])
samplemerged['TrialID']

# plot to check
plt.plot(samplemerged['time'], samplemerged['f2_clean'])
plt.show()